# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Heart-Disease-Predection'

experiment=Experiment(ws, experiment_name)

In [3]:
#Creating Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
#Create compute cluster


# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')


Found existing compute target


In [4]:
#Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart Failure Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("Found")

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/BMUHAMMDI/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [5]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(df, test_size=0.20,train_size=0.80)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
if not os.path.isdir("data"):
    os.mkdir("data")

pd.DataFrame(training_data).to_csv("data/train_data.csv")
ds=ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='HeartDataset', overwrite=True,show_progress=True)
train_data=TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataset/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 25,
                    "max_concurrent_iterations": 4,
                    "n_cross_validations": 5,
                     "training_data" :train_data,
                    "primary_metric": 'accuracy',
                    "iterations": 10,
                        
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(  
    task='classification',
    compute_target='cpu-cluster',
    label_column_name ='DEATH_EVENT',
    enable_onnx_compatible_models=True,
    **automl_settings
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_f58a8166-7ca0-40fd-9abb-738d30eebcdc

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_automl_run,fitted_model = remote_run.get_output()

In [12]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Heart-Disease-Predection,AutoML_f58a8166-7ca0-40fd-9abb-738d30eebcdc_9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
best_automl_run.get_metrics()

{'AUC_micro': 0.9204488189100146,
 'AUC_weighted': 0.9203646310542861,
 'average_precision_score_macro': 0.8897726452458489,
 'matthews_correlation': 0.7138922329367846,
 'AUC_macro': 0.9203646310542861,
 'f1_score_micro': 0.8747340425531915,
 'precision_score_micro': 0.8747340425531915,
 'accuracy': 0.8747340425531915,
 'recall_score_micro': 0.8747340425531915,
 'precision_score_macro': 0.8682541491709573,
 'balanced_accuracy': 0.8469512710710534,
 'recall_score_weighted': 0.8747340425531915,
 'weighted_accuracy': 0.8950521023898199,
 'average_precision_score_micro': 0.921927754326507,
 'recall_score_macro': 0.8469512710710534,
 'f1_score_weighted': 0.8728098833457463,
 'log_loss': 0.35487074562059534,
 'norm_macro_recall': 0.6939025421421066,
 'average_precision_score_weighted': 0.9172045172017402,
 'f1_score_macro': 0.8506575765528612,
 'precision_score_weighted': 0.8823219537499511,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_f58a8166-7ca0-40fd-9abb-738d30eebcdc_

In [14]:
#TODO: Save the best model
best_automl_run,onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model,file_path="./automl_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
#Register the best automl model,
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [16]:
script_file_name ='inference/sscore.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py','inference/sscore.py')

In [17]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_automl_run.get_environment())

In [30]:

#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9001)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)

#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "heart-failure-model",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a
 ---> c354c32ff2d1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 62cf59afc481
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImFhN2NmOGU4LWQyM2YtNGJjZS1hN2I5LTFmMGI0ZTBhYzhlZSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNDQ1MSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNDQ1MSIsIndvcmtzcGFjZUlkIjoiYzQwMTA1NWMtMWI0Mi00NTk4LTkzYjktMzQ3MDMzOTc0YjFjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in c79e70c9d49e
 ---> 26ffd4e0f5a5
Step 4/5 : RUN mv '/var/azureml-app/tmpmsxnfudx.py' /var/azureml-app/main.py
 ---> Running in da0b7d46c0e0
 ---> 2a06463bbee2
Step 5/5 : 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
pd.DataFrame(test_data).to_csv("data/test_ds.csv")
test_ds=pd.read_csv("data/test_ds.csv")
#test_ds.drop(test_ds.columns[0],axis=1,inplace=True)
#test_ds.drop(test_ds.columns[12],axis=1,inplace=True)
test_ds

,Unnamed: 0,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,247,64.00,0,143,0,25,0,246000.00,2.40,135,1,0,214,0
1,79,55.00,0,336,0,45,1,324000.00,0.90,140,0,0,74,0
2,204,78.00,1,64,0,40,0,277000.00,0.70,137,1,1,187,0
3,25,80.00,0,148,1,38,0,149000.00,1.90,144,1,1,23,1
4,163,50.00,1,2334,1,35,0,75000.00,0.90,142,0,0,126,1
5,234,53.00,1,582,0,45,0,305000.00,1.10,137,1,1,209,0
6,283,65.00,0,1688,0,38,0,263358.03,1.10,138,1,1,250,0
7,179,55.00,0,835,0,40,0,279000.00,0.70,140,1,1,147,0
8,31,85.00,0,23,0,45,0,360000.00,3.00,132,1,0,28,1
9,98,60.00,1,156,1,25,1,318000.00,1.20,137,0,0,85,0


In [39]:
data= test_ds[0:3].to_json(orient="table", index=False)
test_sample=json.dumps(json.loads(data)["data"],indent=4)
output=service.run(test_sample)
print(output)

ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '68', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 12 Jan 2021 15:28:09 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '01c5c44f-74ad-440c-bda2-936f91f92edc', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'FunctionWrapper object argument after ** must be a mapping, not list'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '68', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 12 Jan 2021 15:28:09 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '01c5c44f-74ad-440c-bda2-936f91f92edc', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'FunctionWrapper object argument after ** must be a mapping, not list'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '68', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 12 Jan 2021 15:28:09 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '01c5c44f-74ad-440c-bda2-936f91f92edc', '

TODO: In the cell below, print the logs of the web service and delete the service

In [40]:
#Print the webservice logs
print(service.get_logs())

2021-01-12T15:05:58,523384000+00:00 - iot-server/run 
2021-01-12T15:05:58,537106000+00:00 - gunicorn/run 
2021-01-12T15:05:58,542058400+00:00 - rsyslog/run 
2021-01-12T15:05:58,579883400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
#Delete the service
service.delete()